In [20]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet import ResNet50

In [4]:

#data = pd.read_csv('/datasets/faces/labels.csv')

In [12]:
#datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25, vertical_flip=True, horizontal_flip=True)



def load_train(path):
    train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25, vertical_flip=True, horizontal_flip=True)
    train_datagen_flow = train_datagen.flow_from_dataframe(
        dataframe=pd.read_csv(path + 'labels.csv'),
        directory=path + 'final_files',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345,
        subset='training'
    )
    
    return train_datagen_flow

def load_test(path):
    test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25, vertical_flip=True, horizontal_flip=True)
    test_datagen_flow = test_datagen.flow_from_dataframe(
        dataframe=pd.read_csv(path + 'labels.csv'),
        directory=path + 'final_files',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345,
        subset='validation'
    )
    
    return test_datagen_flow

In [18]:
def create_model(input_shape):
    optimizer = Adam(0.0001)
    backbone = ResNet50(input_shape=input_shape, weights='/datasets/keras_models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False)
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='relu')) 
    #model.add(Conv2D, filters=4, kernel_size=(4, 4), padding='same', activation='relu', input_shape=input_shape)
    #model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
    #model.add.Flatten()
    #model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae']) #[tensorflow.keras.metrics.MeanAbsoluteError()])
    return model



In [19]:
def train_model(model, train_data, test_data, batch_size=None, epochs=20,
               steps_per_epoch=None, validation_steps=None):
    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
        if validation_steps is None:
            validation_steps = len(test_data)
    model.fit(train_data, 
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=False)

    return model